In [1]:
import sys
sys.path.append(R'E:\Data science\Titanic dataset\notebooks')

from auto_imports import model_results_imports ,auto_imports

model_results_imports()
auto_imports()

import joblib  # to laod the models
predict_data = pd.read_csv('E:\Data science\Titanic dataset\data\Processed data\Data Modeling\predicting_data.csv')

<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
C:\Users\MinaAlber\AppData\Local\Temp\ipykernel_9952\3574336889.py:10: SyntaxWarning: invalid escape sequence '\D'
  predict_data = pd.read_csv('E:\Data science\Titanic dataset\data\Processed data\Data Modeling\predicting_data.csv')


In [2]:
# loading the models
logit = joblib.load('E:\Data science\Titanic dataset\Models\LogisticRegression.pkl')
RF_ = joblib.load('E:\Data science\Titanic dataset\Models\RandomForest.pkl')
svm_ = joblib.load('E:\Data science\Titanic dataset\Models\SupportVectorMachine.pkl')

I will make ensembling manually becuase of thresholds

### Ensemble

In [3]:
# getting probabilities 
svm_probs = svm_.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]
RF_probs = RF_.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]
logit_probs = logit.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]


# get predictions
svm_preds = (svm_probs >= 0.4).astype(int)
logit_preds = (logit_probs >= 0.6).astype(int)
RF_preds = (RF_probs >= 0.4).astype(int)

In [4]:
# Majority voting manually
ensemble_preds = ((svm_preds + logit_preds + RF_preds) >= 2).astype(int)

In [5]:
ensemble_preds[:5]

array([0, 1, 0, 0, 1])

In [6]:
pd.DataFrame(
    {
        'PassengerId':predict_data['PassengerId'],
        'Survived': ensemble_preds
    }
).to_csv(r'E:\Data science\Titanic dataset\data\Processed data\final_predictions.csv' ,index=False )